In [1]:
import csv
import pandas as pd
import numpy as np
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.bigquery import magics
magics.context.use_bqstorage_api = True
import pyarrow
from sklearn.linear_model import LogisticRegression
from matplotlib import pyplot as plt
import random
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
np.set_printoptions(suppress=True, linewidth=200, edgeitems=100)

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

In [2]:
import os
BIGQUERY_PROJECT = 'cloud-test-ironhacks'
BIGQUERY_KEYPATH = 'cloud-test-ironhacks-052e1ced8b54.json'

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = BIGQUERY_KEYPATH
bigquery_client = bigquery.Client(project=BIGQUERY_PROJECT)

In [3]:
query = """
SELECT uu_id, week_number, total_claims, edu_post_hs
FROM cloud-test-ironhacks.dwd_data.suppression_remove
"""
query_job = bigquery_client.query(query)
data = query_job.to_dataframe()
data.head()

,uu_id,week_number,total_claims,edu_post_hs
0,9bee42b55f891413ae7fac2d9d89280a,1,30.0,NaN
1,9bee42b55f891413ae7fac2d9d89280a,2,51.0,20.0
2,9bee42b55f891413ae7fac2d9d89280a,3,39.0,NaN
3,9bee42b55f891413ae7fac2d9d89280a,4,36.0,15.0
4,9bee42b55f891413ae7fac2d9d89280a,5,49.0,15.0


In [4]:
data.isnull().any()

uu_id           False
week_number     False
total_claims     True
edu_post_hs      True
dtype: bool

In [5]:
data = data.fillna(lambda x: x.median())

In [10]:
data['edu_post_hs'] = data['edu_post_hs'].fillna(data['edu_post_hs'].mean())
data['total_claims'] = data['total_claims'].fillna(data['total_claims'].mean())
data['total_claims'] = pd.to_numeric(data['total_claims'], errors='coerce')
data['edu_post_hs'] = pd.to_numeric(data['edu_post_hs'], errors='coerce')

In [11]:
data

,uu_id,week_number,total_claims,edu_post_hs
0,9bee42b55f891413ae7fac2d9d89280a,1,30.0,18.662311
1,9bee42b55f891413ae7fac2d9d89280a,2,51.0,20.000000
2,9bee42b55f891413ae7fac2d9d89280a,3,39.0,18.662311
3,9bee42b55f891413ae7fac2d9d89280a,4,36.0,15.000000
4,9bee42b55f891413ae7fac2d9d89280a,5,49.0,15.000000
...,...,...,...,...
57367,20f6ad2e745b760f4d67491c4c9f2881,32,0.0,0.000000
57368,20f6ad2e745b760f4d67491c4c9f2881,33,0.0,0.000000
57369,20f6ad2e745b760f4d67491c4c9f2881,34,0.0,0.000000
57370,20f6ad2e745b760f4d67491c4c9f2881,35,0.0,0.000000


In [12]:
X = data[['week_number', 'edu_post_hs']]
y = data['total_claims']
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.2)

In [15]:
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier()
model.fit(X_train, y_train)

GradientBoostingClassifier()

In [16]:
y_pred = model.predict(X_test)

In [20]:
y_test = model.predict([[37, 10]])
y_test

array([19.])